In [1]:
def infoGain(P, N):  #Claculate Information gain or Class entropy
    import math
    return -P / (P + N) * math.log2(P / (P + N)) - N / (P + N) * math.log2(N / (P + N))

In [2]:
def insertNode(tree, addTo, Node):
    for k, v in tree.items():  #Traversal of tree
        if isinstance(v, dict):
            tree[k] = insertNode(v, addTo, Node)
    if addTo in tree:  #If d is found then add Node
        if isinstance(tree[addTo], dict):
            tree[addTo][Node] = 'None'
        else:
            tree[addTo] = {Node: 'None'}
    return tree

In [3]:
def insertConcept(tree, addTo, Node):
    for k, v in tree.items():  #Traversal of tree
        if isinstance(v, dict):
            tree[k] = insertConcept(v, addTo, Node)
    if addTo in tree:  #If d is found then add Node
        tree[addTo] = Node
    return tree

In [4]:
def getNextNode(data, AttributeList, concept, conceptVals, tree, addTo):
    Total = data.shape[0]
    if Total == 0:  #If Attibutes are empty, then return current value of tree
        return tree

    countC = {}
    for cVal in conceptVals:  #If Example is positive then return positive and If negative then return negative
        dataCC = data[data[concept] == cVal]  #Get data for specific concept
        countC[cVal] = dataCC.shape[0]  #Get the count of data for specific concept

    if countC[conceptVals[0]] == 0:  #If all examples are Positive(Not Negative), return single node Positive
        tree = insertConcept(tree, addTo, conceptVals[1])
        return tree
    if countC[conceptVals[1]] == 0:  #If all examples are Negative (Not Positive), return single node Negative
        tree = insertConcept(tree, addTo, conceptVals[0])
        return tree

    ClassEntropy = infoGain(countC[conceptVals[0]], countC[conceptVals[1]])  #Calculate Class Entropy for data

    Attr = {}  #Attribute dict holding list of possible values
    for a in AttributeList:
        Attr[a] = list(set(data[a]))

    AttrCount = {}  #Get the attribute values being positive and negative
    EntropyAttr = {}  #Attribute Entropy
    for att in Attr:
        for vals in Attr[att]:
            for c in conceptVals:
                iData = data[data[att] == vals]  #Get data for specific attribute
                dataAtt = iData[iData[concept] == c]  #Get data for specific attribute and concept
                AttrCount[c] = dataAtt.shape[0]  #Get the count of data for specific attribute and concept
            TotalInfo = AttrCount[conceptVals[0]] + AttrCount[conceptVals[1]]  #Total Attribute
            if AttrCount[conceptVals[0]] == 0 or AttrCount[conceptVals[1]] == 0:
                InfoGain = 0
            else:
                InfoGain = infoGain(AttrCount[conceptVals[0]], AttrCount[conceptVals[1]])  #Calculate InfoGain for each  attr

            if att not in EntropyAttr:  #Calculate Entropy for each attr
                EntropyAttr[att] = (TotalInfo / Total) * InfoGain
            else:
                EntropyAttr[att] = EntropyAttr[att] + (TotalInfo / Total) * InfoGain

    Gain = {}
    for g in EntropyAttr:
        Gain[g] = ClassEntropy - EntropyAttr[g]  #Calculate gain

    Node = max(Gain, key=Gain.get)  #Get root node

    tree = insertNode(tree, addTo, Node)  #Add Node to tree
    for nD in Attr[Node]:
        tree = insertNode(tree, Node, nD)  #Insert Attribute value to tree
        newData = data[data[Node] == nD].drop(Node, axis=1)  #Get new data with Attribute value nD and removing rows with column value Node
        AttributeList = list(newData)[:-1]  #New Attribute List
        tree = getNextNode(newData, AttributeList, concept, conceptVals, tree, nD)  #Call the function recursively
    return tree

In [5]:
import pandas as pd
def main():
    data = pd.read_csv('Datasets/PlayTennis.csv')  #Read CSV
    if 'Unnamed: 0' in data.columns:
        data = data.drop('Unnamed: 0', axis=1)
#     data = data.drop('slno', axis=1)
    AttributeList = list(data)[:-1]  #Get Attribute List
    concept = str(list(data)[-1])  #Get concept list
    conceptVals = list(set(data[concept]))  #Get Concept values
    tree = getNextNode(data, AttributeList, concept, conceptVals,
                       {'root': 'None'}, 'root')
    return tree  #Call recursive function with initial value of tree and Node as root

In [6]:
tree = main()['root']

In [7]:
df = pd.read_csv('Datasets/PlayTennis.csv')


def test(tree, d):
    for k in tree:
        for v in tree[k]:
            if (d[k] == v and isinstance(tree[k][v], dict)):
                test(tree[k][v], d)
            elif (d[k] == v):
                print("Classification: " + tree[k][v])

In [8]:
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)
df.head()


,Outlook,Temperature,Humidity,Wind,PlayTennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [9]:
print(tree)

{'Outlook': {'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}, 'Sunny': {'Humidity': {'Normal': 'Yes', 'High': 'No'}}, 'Overcast': 'Yes'}}


In [10]:
test(tree, df.loc[0, :])

Classification: No
